# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for plotting the spectrogram for a selected seismogram <br/>

- details regarding the spectrogram plotter are
[here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.specgram.html)

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import os
import sys
import warnings

from obspy import read
from lib_seis import station_map_and_table

In [ ]:
# script settings

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = 9, 6

In the subsequent code cell, select the *example_index* from the following list for the example you want to
view seismograms and spectrograms for -

    1   - Yahtse glacier calving event
    2   - Mw 7.5 earthquake in southeastern Alaska
    3   - explosion in Fairbanks
    4   - very low frequency (VLF) event near Kantishna
    5   - landslide near Lituya Bay
    6   - Mw 8.6 Indian Ocean (offshore Sumatra) recorded in Alaska, plus triggered earthquakes
    6.1 - triggered earthquake (Andreanof)
    6.2 - triggered earthquake (Nenana crustal)
    6.3 - triggered earthquake (Iliamna intraslab)
    7   - your own example

>__NOTE:__ The notebook requires that the user has already run lab_record_section.ipynb and downloaded the
> necessary data corresponding to each example.

In [ ]:
example_index = 1                                                                # select example

In [ ]:
# plot source station map for the selected example

print('\nplotting source station map ....\n')
plt.figure()
data_dir = f'datawf/Example_{example_index}'
source_station_map = img.imread(f'{data_dir}/station_map.png')
plt.imshow(source_station_map)
plt.show()

In [ ]:
# station distances and azimuth table

event_path = f'{data_dir}/event.xml'
inv_path = f'{data_dir}/inv.xml'
station_map_and_table(event_path, inv_path, print_map=False)

Examine the map and table and select a network, station, location and channel code to pick a particular
seismogram.

In [ ]:
# by default, this will load ALL saved files for your example
# alternatively, you can either specify a specific subset here -- or use ipick (see below)
network  = 'XF'                                                                  # select network
station  = 'BOOM'                                                                # select station
location = ''                                                                    # select location
channel  = 'HHZ'                                                                 # select channel

In [ ]:
# load sac files and list them
sac_file = f'Example_{example_index}.{network}.{station}.{location}.{channel}.sac'

if not os.path.exists(f'{data_dir}/{sac_file}'):
    sys.exit(f'check the network, station, location and channel inputs; sac file {sac_file} does not exist.')
else:
    st = read(f'{data_dir}/{sac_file}', 'SAC')

tr = st[0]

t                  = tr.times()
data               = tr.data
sampling_frequency = tr.stats.sampling_rate

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8,6))
ax1.plot(t, data, 'k', linewidth=0.5)

image = ax2.specgram(tr, Fs=sampling_frequency, noverlap=int(0.8*256), cmap="jet")
ax2.set_xlabel('Time - Seconds')
ax2.set_ylabel('Frequency (Hz)')

ax3 = fig.add_axes([0.9, 0.1, 0.03, 0.37])
plt.colorbar(mappable=image[3], cax=ax3)
plt.ylabel('Relative Amplitude (dB)')

fig.suptitle(f'{tr.id} − starting {tr.stats["starttime"]}')
plt.show()

# note: the time axis of the spectrogram may not correspond to the time axis of the record sections
# this is because the spectrogram plotter assigns a default value of time = 0 to the first sample of the input
# data

if example_index == 1:
    fig.canvas.draw()
    labels = np.arange(-40,100,20)
    ax2.set_xticklabels(labels)